In [3]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cce79d89-f0d7-45f0-a07e-8290776ba420)

In [4]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

#import the model and configure input and output columns

model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error")
        )

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 6, Finished, Available, Finished)

In [5]:
#Apply the model to our dataframe

result = model.transform(df)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 7, Finished, Available, Finished)

In [6]:
display(result)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b53d296-9cb5-4da9-b354-6359724019fd)

In [7]:
#Create a sentiment column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment",col("response.documents.sentiment"))

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 9, Finished, Available, Finished)

In [8]:
display(sentiment_df)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 69e8ada7-a6e4-4553-8246-7d09e4119819)

In [9]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 11, Finished, Available, Finished)

In [10]:
display(sentiment_df_final)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b2f2a436-d3da-4748-a248-3dae34898a6d)

In [23]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished",to_date(col("datePublished"),"dd-MMM-yyyy"))

display(sentiment_df_final)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c089fa4b-b085-4c70-9ac8-d8eed473ed1b)

In [12]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:
    print("Table already exists.")  

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")
    
    spark.sql(f""" MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url
                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *
                    """)

     

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 14, Finished, Available, Finished)

Table already exists.


In [13]:
display(sentiment_df_final)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bd0c0ed4-0e9c-4a5c-9906-4cbe608d6152)

In [14]:
#Temporary Code - Delete After Running

df = spark.sql("SELECT * FROM bing_lake_db.tbl_sentiment_analysis") 
display(df)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ad44a704-5ee4-4efc-b987-900b1a77b06f)

In [15]:
#from pyspark.sql.functions import col, to_date

#df = df.withColumn("datepublished",to_date(col("datepublished"),"dd-MMM-yyy"))

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 17, Finished, Available, Finished)

In [16]:
#display(df)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 18, Finished, Available, Finished)

In [17]:
#df.printSchema()

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 19, Finished, Available, Finished)

In [18]:
#df.write.format('delta').mode("overwrite").option("overwriteSchema","True").saveAsTable(table_name)

StatementMeta(, 70df4dfa-098a-4a50-86c3-e9696eaa8c5e, 20, Finished, Available, Finished)